In [1]:
# import all libraries etc.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
import statsmodels.api as sm
import statsmodels.stats.api as sms
import scipy.stats as stats
%matplotlib inline
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
import warnings

In [2]:
pd.set_option('display.max_columns', 300)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv("data/Xtrain.csv")
y = pd.read_csv('data/ytrain.csv')

test = pd.read_csv('data/Xtest.csv')

In [ ]:
#data['price'] = y

In [6]:
######for train dataset
sqft_basement_list= []
for num in data['sqft_basement'].values:
    if num == '?':
        sqft_basement_list.append(np.nan)
    else:
        sqft_basement_list.append(float(num))
data['sqft_basement']= sqft_basement_list
#cleaning year renovated        
data['yr_renovated'] = data['yr_renovated'].fillna(0).astype(int)
data['yr_renovated'] = np.where(data['yr_renovated'] == 0, data['yr_built'], data['yr_renovated'])

#waterfron cleaning
data['waterfront'].replace(np.nan, 0, inplace =True)

#view col cleaning
data['view'].replace(np.nan, 0 , inplace =True)

#basement
data['sqft_basement'].fillna(0, inplace= True)

#one house has a 33 bedroom house to 3:
data['bedrooms'].replace(33, 3, inplace=True)





######for test data
sqft_basement_list2= []
for num in test['sqft_basement'].values:
    if num == '?':
        sqft_basement_list2.append(np.nan)
    else:
        sqft_basement_list2.append(float(num))
test['sqft_basement']= sqft_basement_list2
#cleaning year renovated        
test['yr_renovated'] = test['yr_renovated'].fillna(0).astype(int)
test['yr_renovated'] = np.where(test['yr_renovated'] == 0, test['yr_built'], test['yr_renovated'])


#waterfron cleaning
test['waterfront'].replace(np.nan, 0, inplace =True)
#view col cleaning
test['view'].replace(np.nan, 0 , inplace =True)

#basement
test['sqft_basement'].fillna(0, inplace= True)

#drop not useful cols such as ID, Date
#data.drop(['id', 'date'],1, inplace= True)

#one house has a 33 bedroom house to 3:
test['bedrooms'].replace(33, 3, inplace=True)


In [ ]:
#top25 zipcode

# zip_code = data.groupby("zipcode")["price"].mean().sort_values(ascending = False)
# zip_code.describe()

# top_20 = data.groupby("zipcode")["price"].mean().sort_values(ascending = False)[:20]
# top_20

In [ ]:
##zipcode_index = top_20.index

In [ ]:
##data['top20_neighborhood'] = data['zipcode'].isin(zipcode_index).astype(int)

In [7]:
#########Dummying grade for train###########
grade_origin= data['grade']
df_dummy_grade= pd.get_dummies(grade_origin, drop_first= True) #avoid dummy trap
data = pd.concat([df_dummy_grade, data], axis=1)

#renaming the new Columns
#grade
mapper = {4:'Grade_4', 5: 'Grade_5', 6:'Grade_6',7:'Grade_7',
          8:'Grade_8',9:'Grade_9',10:'Grade_10', 11:'Grade_11',12:'Grade_12',13:'Grade_13'} 
data.rename(mapper=mapper, axis=1, inplace= True)




############Dummying grade for test###########
grade_origin2= test['grade']
df_dummy_grade2= pd.get_dummies(grade_origin2, drop_first= False) #avoid dummy trap
test = pd.concat([df_dummy_grade2, test], axis=1)

#renaming the new Columns
#grade
mapper2 = {4:'Grade_4', 5: 'Grade_5', 6:'Grade_6',7:'Grade_7',
          8:'Grade_8',9:'Grade_9',10:'Grade_10', 11:'Grade_11',12:'Grade_12',13:'Grade_13'} 
test.rename(mapper=mapper2, axis=1, inplace= True)

In [8]:
test

,Grade_4,Grade_5,Grade_6,Grade_7,Grade_8,Grade_9,Grade_10,Grade_11,Grade_12,Grade_13,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,0,1,0,0,0,0,0,0,0,2/20/2015,3,0.75,850,8573,1.00,0.00,0.00,3,6,600,250.00,1945,1945,98146,47.50,-122.36,850,8382
1,0,0,1,0,0,0,0,0,0,0,10/8/2014,3,1.00,1510,6083,1.00,0.00,0.00,4,6,860,650.00,1940,1940,98115,47.70,-122.32,1510,5712
2,0,0,0,1,0,0,0,0,0,0,3/25/2015,4,2.25,1790,42000,1.00,0.00,0.00,3,7,1170,620.00,1983,1983,98045,47.48,-121.74,2060,50094
3,0,0,0,1,0,0,0,0,0,0,2/17/2015,2,1.50,1140,2500,1.00,0.00,1.00,3,7,630,510.00,1988,1988,98106,47.57,-122.36,1500,5000
4,0,0,0,1,0,0,0,0,0,0,5/23/2014,3,1.00,1500,3920,1.00,0.00,0.00,3,7,1000,500.00,1947,1947,98107,47.67,-122.36,1640,4017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,0,0,1,0,0,0,0,0,0,0,6/27/2014,5,1.00,1170,6757,1.00,0.00,0.00,4,6,800,370.00,1944,1944,98125,47.73,-122.30,1590,6794
5396,0,0,0,1,0,0,0,0,0,0,1/20/2015,3,1.75,1670,5100,1.00,0.00,2.00,5,7,990,680.00,1954,1954,98144,47.59,-122.29,2140,4452
5397,0,0,0,0,0,1,0,0,0,0,3/24/2015,4,2.25,3260,4640,2.00,0.00,0.00,5,9,2360,900.00,1907,1907,98112,47.63,-122.31,3240,5800
5398,0,0,0,0,0,0,0,1,0,0,3/16/2015,4,5.00,5820,13906,2.00,0.00,0.00,3,11,3750,2070.00,1993,1993,98042,47.38,-122.16,2980,13000


In [9]:
data

,Grade_4,Grade_5,Grade_6,Grade_7,Grade_8,Grade_9,Grade_10,Grade_11,Grade_12,Grade_13,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,0,0,0,1,0,0,0,0,0,3/4/2015,3,2.50,1880,4499,2.00,0.00,0.00,3,8,1880,0.00,1993,1993,98029,47.57,-122.00,2130,5114
1,0,0,0,1,0,0,0,0,0,0,10/7/2014,3,2.50,2020,6564,1.00,0.00,0.00,3,7,1310,710.00,1994,1994,98042,47.35,-122.16,1710,5151
2,0,0,0,0,0,1,0,0,0,0,1/16/2015,5,4.00,4720,493534,2.00,0.00,0.00,5,9,3960,760.00,1975,1975,98027,47.45,-122.01,2160,219542
3,0,0,0,1,0,0,0,0,0,0,3/30/2015,2,2.00,1430,3880,1.00,0.00,0.00,4,7,1430,0.00,1949,1949,98117,47.68,-122.39,1430,3880
4,0,0,0,0,1,0,0,0,0,0,10/14/2014,3,2.25,2270,32112,1.00,0.00,0.00,4,8,1740,530.00,1980,1980,98042,47.35,-122.09,2310,41606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16192,0,0,0,1,0,0,0,0,0,0,9/15/2014,3,2.50,2230,5800,2.00,0.00,0.00,3,7,2230,0.00,2004,2004,98065,47.53,-121.85,2230,6088
16193,0,0,0,0,1,0,0,0,0,0,10/2/2014,4,2.75,2770,3852,2.00,0.00,0.00,3,8,2770,0.00,2014,2014,98178,47.50,-122.23,1810,5641
16194,0,0,1,0,0,0,0,0,0,0,7/21/2014,4,1.50,1530,9000,1.00,0.00,0.00,4,6,1530,0.00,1976,1976,98014,47.65,-121.91,1520,8500
16195,0,1,0,0,0,0,0,0,0,0,6/20/2014,1,0.75,380,15000,1.00,0.00,0.00,3,5,380,0.00,1963,1963,98168,47.48,-122.32,1170,15000


In [10]:
#making dummy for condition for train
condition_origin= data['condition']
df_cond_dum=pd.get_dummies(condition_origin, drop_first=True)
data= pd.concat([df_cond_dum, data], axis=1)

mapper3 = {1:'Condition_1',2:'Condition_2',3:'Condition_3',4:'Condition_4',5:'Condition_5'}
data.rename(mapper=mapper3, axis=1, inplace=True)



#dummy condition for test
condition_origin2= test['condition']
df_cond_dum2=pd.get_dummies(condition_origin2, drop_first=True)
test= pd.concat([df_cond_dum2, test], axis=1)

mapper3 = {1:'Condition_1',2:'Condition_2',3:'Condition_3',4:'Condition_4',5:'Condition_5'}
test.rename(mapper=mapper3, axis=1, inplace=True)

In [11]:
# adding some interactions
data['age'] = 2021 - data['yr_built']

#decided to combine Bath and Bedroom into a ratio (Bed:Bath) since its a commonly used metric fpr real estate

data['bed:bath'] = data['bedrooms']/data['bathrooms']

data = data.loc[:,~data.columns.duplicated()]

In [12]:
#Dummying the zip train
zip_origin= data['zipcode']
df_zip= pd.get_dummies(zip_origin, drop_first= True)
data = pd.concat([df_zip, data], axis=1)

data = data.drop('zipcode',axis = 1)





#dummy the zip test
zip_origin2= test['zipcode']
df_zip2= pd.get_dummies(zip_origin2, drop_first= True)
test = pd.concat([df_zip2, test], axis=1)

test = test.drop('zipcode',axis = 1)




#renaming the new Columns
#view

#mapper_view = {4.0:'View 4', 2.0: 'View 2', 3.0:'View 3',1.0:'View 1'}

#data.rename(mapper=mapper_view, axis=1, inplace= True)

In [13]:
# adding some interactions train
data['age'] = 2021 - data['yr_built']

#decided to combine Bath and Bedroom into a ratio (Bed:Bath) since its a commonly used metric fpr real estate

data['bed:bath'] = data['bedrooms']/data['bathrooms']

data = data.loc[:,~data.columns.duplicated()]




# adding some interactions test
test['age'] = 2021 - test['yr_built']

#decided to combine Bath and Bedroom into a ratio (Bed:Bath) since its a commonly used metric fpr real estate

test['bed:bath'] = test['bedrooms']/test['bathrooms']

test = test.loc[:,~test.columns.duplicated()]

In [14]:
#drop test
data.drop(['view', 'grade', 'bedrooms', 'bathrooms', 'condition', 'yr_renovated'], axis=1, inplace=True)

#drop train
test.drop(['view', 'grade', 'bedrooms', 'bathrooms', 'condition', 'yr_renovated'], axis=1, inplace=True)

In [ ]:
#rmv_outlier(data,'price')

In [ ]:
test

In [ ]:
data

In [15]:

#drop test
data = data.drop(['Grade_13', 'sqft_above', "yr_built",'date'], axis=1)

#drop train
test = test.drop(['Grade_13', 'sqft_above', "yr_built",'date'], axis=1)


In [16]:
data

,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,98019,98022,98023,98024,98027,98028,98029,98030,98031,98032,98033,98034,98038,98039,98040,98042,98045,98052,98053,98055,98056,98058,98059,98065,98070,98072,98074,98075,98077,98092,98102,98103,98105,98106,98107,98108,98109,98112,98115,98116,98117,98118,98119,98122,98125,98126,98133,98136,98144,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199,Condition_2,Condition_3,Condition_4,Condition_5,Grade_4,Grade_5,Grade_6,Grade_7,Grade_8,Grade_9,Grade_10,Grade_11,Grade_12,sqft_living,sqft_lot,floors,waterfront,sqft_basement,lat,long,sqft_living15,sqft_lot15,age,bed:bath
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1880,4499,2.00,0.00,0.00,47.57,-122.00,2130,5114,28,1.20
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,2020,6564,1.00,0.00,710.00,47.35,-122.16,1710,5151,27,1.20
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,4720,493534,2.00,0.00,760.00,47.45,-122.01,2160,219542,46,1.25
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1430,3880,1.00,0.00,0.00,47.68,-122.39,1430,3880,72,1.00
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,2270,32112,1.00,0.00,530.00,47.35,-122.09,2310,41606,41,1.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16192,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,2230,5800,2.00,0.00,0.00,47.53,-121.85,2230,6088,17,1.20
16193,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,2770,3852,2.00,0.00,0.00,47.50,-122.23,1810,5641,7,1.45
16194,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1530,9000,1.00,0.00,0.00,47.65,-121.91,1520,8500,45,2.67
16195,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,380,15000,1.00,0.00,0.00,47.48,-122.32,1170,15000,58,1.33


In [17]:
test


,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,98019,98022,98023,98024,98027,98028,98029,98030,98031,98032,98033,98034,98038,98039,98040,98042,98045,98052,98053,98055,98056,98058,98059,98065,98070,98072,98074,98075,98077,98092,98102,98103,98105,98106,98107,98108,98109,98112,98115,98116,98117,98118,98119,98122,98125,98126,98133,98136,98144,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199,Condition_2,Condition_3,Condition_4,Condition_5,Grade_4,Grade_5,Grade_6,Grade_7,Grade_8,Grade_9,Grade_10,Grade_11,Grade_12,sqft_living,sqft_lot,floors,waterfront,sqft_basement,lat,long,sqft_living15,sqft_lot15,age,bed:bath
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,850,8573,1.00,0.00,250.00,47.50,-122.36,850,8382,76,4.00
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1510,6083,1.00,0.00,650.00,47.70,-122.32,1510,5712,81,3.00
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1790,42000,1.00,0.00,620.00,47.48,-121.74,2060,50094,38,1.78
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1140,2500,1.00,0.00,510.00,47.57,-122.36,1500,5000,33,1.33
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1500,3920,1.00,0.00,500.00,47.67,-122.36,1640,4017,74,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1170,6757,1.00,0.00,370.00,47.73,-122.30,1590,6794,77,5.00
5396,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1670,5100,1.00,0.00,680.00,47.59,-122.29,2140,4452,67,1.71
5397,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,3260,4640,2.00,0.00,900.00,47.63,-122.31,3240,5800,114,1.78
5398,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,5820,13906,2.00,0.00,2070.00,47.38,-122.16,2980,13000,28,0.80


In [18]:
x_train = data
y

,price
0,529000.00
1,253000.00
2,745000.00
3,545000.00
4,390000.00
...,...
16192,440000.00
16193,572000.00
16194,299800.00
16195,245000.00


In [19]:
lr = LinearRegression()

In [20]:
scores_simple = cross_validate(
                    lr, x_train, y, cv=5, 
                    return_train_score=True)

In [21]:
lr.fit(x_train, y)
print(lr.score(x_train, y))

0.8261355890673305


In [22]:
y_hat_train = lr.predict(x_train)

In [23]:
train_mse = mean_squared_error(y, y_hat_train, squared =False)
print('Train Mean Error:', train_mse)

Train Mean Error: 152750.24369515275


In [104]:
y_hat_test = lr.predict(test)

In [105]:
y_hat_test

array([[ 161969.66073395],
       [ 540938.07934476],
       [ 340080.70554999],
       ...,
       [1332158.36617207],
       [1349228.88117586],
       [ 178439.84196886]])

In [90]:
import numpy as np
y_test_fake = np.full((5400,1), 0)

In [91]:
from sklearn.metrics import r2_score

# fake predictions using the mean of y_train.
y_hat_test = np.full((5400,1), np.mean(y))

r2_score(y_hat_test, y_test_fake)

0.0

In [92]:
len(y_hat_test)

5400

In [109]:
np.savetxt('Group2_bakeoff.csv', y_hat_test, delimiter=',',header = 'price_estimate')

In [95]:
ls

Bakeoff.ipynb        PROJ2_updated.ipynb  halfway-there.gif
CONTRIBUTING.md      README.md            picture/
Group2_bakeoff.csv   Untitled.ipynb       student.ipynb
LICENSE.md           data/                test-el.ipynb


In [86]:
y_hat_test.shape

(5400, 1)

In [110]:
df = pd.read_csv("Group2_bakeoff.csv")

In [111]:
df

,# price_estimate
0,161969.66
1,540938.08
2,340080.71
3,260249.26
4,533350.71
...,...
5395,343804.91
5396,590011.25
5397,1332158.37
5398,1349228.88
